# PyTorch ONNX Exporter new features and architecture

Jan 2026

![infographics](<pt-onnx-infographics.png>)

## `dynamo=True` is the default

- The New Default: Starting from PyTorch 2.9, the **`dynamo=True`** option is the **default and recommended** way to export models to ONNX.
- Core Shift: It moves away from the older TorchScript-based capture mechanism to a torch.export based modern stack.
- Deprecation Plan: While the TorchScript exporter (dynamo=False) is currently usable, it is planned for eventual deprecation in alignment with PyTorch core's handling of TorchScript.

## New options in `export()`

```py
torch.onnx.export(
    model, args, kwargs=kwargs,
    # New way of expressing dynamic shapes (more examples later)
    dynamic_shapes=({0: "batch", 1: "sequence_len"}),
    # dynamic_axes=...,  # Deprecated
    dynamo=True,  # Default (2.9)
    report=True,  # Creates a markdown report
    verify=True,  # Runs onnx runtime on the example
    optimize=True, # Runs onnxscript graph optimizations
) -> torch.onnx.ONNXProgram
```

## What happens inside `torch.onnx.export`

torch.export() **captures FX** graph
-> **translate** and build ONNX IR
-> graph **optimization** with ONNX Script

Entry point is at: https://github.com/pytorch/pytorch/blob/0ad306cac740eaf2ce582e2bdf097cc61d929a40/torch/onnx/_internal/exporter/_core.py#L1282

![diagram](https://raw.githubusercontent.com/justinchuby/diagrams/refs/heads/main/pytorch/torch-export-flow.svg)

In [ ]:
## FX graph and the ExportedProgram

In [28]:
import torch
import torch.export

class Mod(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = torch.nn.Parameter(torch.randn(2, 3, 10, 10))

    def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        a = torch.sin(x)
        a.add_(y)
        b = a * self.weight
        return torch.nn.functional.scaled_dot_product_attention(b, b, b)

example_args = (torch.randn(2, 3, 10, 10), torch.randn(2, 3, 10, 10))

# Important to set to eval mode before exporting
mod = Mod().eval()
exported_program: "ExportedProgram" = torch.export.export(mod, args=example_args)
print(exported_program)

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, p_weight: "f32[2, 3, 10, 10]", x: "f32[2, 3, 10, 10]", y: "f32[2, 3, 10, 10]"):
             # File: /tmp/ipykernel_246584/197873441.py:10 in forward, code: a = torch.sin(x)
            sin: "f32[2, 3, 10, 10]" = torch.ops.aten.sin.default(x);  x = None
            
             # File: /tmp/ipykernel_246584/197873441.py:11 in forward, code: a.add_(y)
            add_: "f32[2, 3, 10, 10]" = torch.ops.aten.add_.Tensor(sin, y);  sin = y = None
            
             # File: /tmp/ipykernel_246584/197873441.py:12 in forward, code: b = a * self.weight
            mul: "f32[2, 3, 10, 10]" = torch.ops.aten.mul.Tensor(add_, p_weight);  add_ = p_weight = None
            
             # File: /tmp/ipykernel_246584/197873441.py:13 in forward, code: return torch.nn.functional.scaled_dot_product_attention(b, b, b)
            scaled_dot_product_attention: "f32[2, 3, 10, 10]" = torch.ops.aten.scaled_dot_product_at

In [29]:
decomposed = exported_program.run_decompositions()
print(decomposed)

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, p_weight: "f32[2, 3, 10, 10]", x: "f32[2, 3, 10, 10]", y: "f32[2, 3, 10, 10]"):
             # File: /tmp/ipykernel_246584/197873441.py:10 in forward, code: a = torch.sin(x)
            sin: "f32[2, 3, 10, 10]" = torch.ops.aten.sin.default(x);  x = None
            
             # File: /tmp/ipykernel_246584/197873441.py:11 in forward, code: a.add_(y)
            add: "f32[2, 3, 10, 10]" = torch.ops.aten.add.Tensor(sin, y);  sin = y = None
            
             # File: /tmp/ipykernel_246584/197873441.py:12 in forward, code: b = a * self.weight
            mul: "f32[2, 3, 10, 10]" = torch.ops.aten.mul.Tensor(add, p_weight);  add = p_weight = None
            
             # File: /tmp/ipykernel_246584/197873441.py:13 in forward, code: return torch.nn.functional.scaled_dot_product_attention(b, b, b)
            mul_1: "f32[2, 3, 10, 10]" = torch.ops.aten.mul.Scalar(mul, 0.5623413251903491)
            

## Translation to ONNX

In [33]:
onnx_program = torch.onnx.export(exported_program, report=True, opset_version=23)
print(onnx_program)

[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
x input_kind: InputKind.USER_INPUT persistent: None
y input_kind: InputKind.USER_INPUT persistent: None
p_weight input_kind: InputKind.PARAMETER persistent: None
[torch.onnx] Translate the graph into ONNX... ✅
[torch.onnx] Export report has been saved to 'onnx_export_2026-01-05_12-44-51-152246_success.md'.
ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 23},
            producer_name='pytorch',
            producer_version='2.10.0.dev20251028+cpu',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"x"<FLOAT,[2,3,10,10]>,
                %"y"<FLOAT,[2,3,10,10]>
            ),
            outputs=(
                %"scaled_dot_product_attention"<FLOAT,[2,3,10,10]>
            ),
            initializers=(
                %"weight"<FL

In [34]:
onnx_program.save("model.onnx")

In [20]:
!onnxvis model.onnx

Loading extensions...
I0000 00:00:1767644749.496075  248247 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

Loaded 9 adapters:
 - TFLite adapter (Flatbuffer)
 - TFLite adapter (MLIR)
 - TF adapter (MLIR)
 - TF adapter (direct)
 - GraphDef adapter
 - Pytorch adapter (exported program)
 - MLIR adapter
 - ONNX adapter
 - JSON adapter

Starting Model Explorer server at:
http://localhost:8080/?data=%7B%22models%22%3A%20%5B%7B%22url%22%3A%20%22/home/justinchu/dev/talk-torch-onnx-apis-architecture/src/model.onnx%22%7D%5D%7D

Press Ctrl+C to stop.
gio: http://localhost:8080/?data=%7B%22models%22%3A%20%5B%7B%22url%22%3A%20%22/home/justinchu/dev/talk-torch-onnx-apis-architecture/src/model.onnx%22%7D%5D%7D: Operation not supported
Stopping server...
^C


## Model in `onnx_program.model` is an onnx_ir.Model

- You can run any ONNX->ONNX transformation on it.
- The exporter by default runs ONNX Script pattern replacement and whole graph optimization. These are robust, in-memory graph passes the team has created
- Low memory consumption by sharing tensor data with the PyTorch model

In [35]:
# Explore the IR model

model = onnx_program.model
print("Model has", len(model.graph), "nodes")

print("All initializers:")
for init in model.graph.initializers.values():
    print(" ", init)

Model has 4 nodes
All initializers:
  %"weight"<FLOAT,[2,3,10,10]>{TorchTensor(...)}


In [36]:
print(model.graph.initializers["weight"].const_value.raw is mod.weight)

True


In [37]:
model.graph.initializers["weight"].const_value.display()

TorchTensor<FLOAT,[2,3,10,10]>(Parameter containing: tensor([[[[-0.6935, -1.0072, -2.4926,  0.3564, -0.9010, 
-0.4989,  0.2919, 1.5580,  0.4942, -0.6712], [ 0.1446,  0.4472, -0.0270, -0.9070, -0.0466, -0.7061,  0.3806, 
0.8480, -0.0855, -0.2280], [-1.1288,  1.3053,  0.8424, -0.5957,  0.6718,  0.4550, -0.7675, -0.2447, -0.6553, 
-0.0908], [ 2.0796,  0.2258,  0.5581, -0.6245, -1.0529,  0.0074,  0.6629, 0.3673, -1.0229,  0.7332], [-1.5294, 
-0.8384, -1.0690, -0.6175,  0.1101, -1.5167, -1.6728, -0.8505, -2.2796, -0.2348], [-0.3038,  0.1262, -0.0177,  
1.5609,  0.6624, -0.9537,  0.5907, -0.1695, -0.4452,  0.5810], [-1.1175, -0.1270, -1.0324, -0.2356,  0.5048,  
0.1460,  0.7469, 0.9473,  0.9916,  0.7179], [-0.7296, -0.0772,  0.3829, -1.2073, -0.1078,  0.1984,  2.1565, 
-0.8187,  0.6580, -0.3323], [ 0.1502, -0.5279, -2.1784,  0.5219,  0.4879,  0.7157, -0.0560, -0.6545, -0.7384,  
0.2617], [ 0.9767, -0.8617, -1.8834, -2.0636, -0.8235,  1.0661, -2.3338, -0.1315, -0.1462, -0.6348]],  [[ 0.6052, 
-0.5105, -0.4847,  1.4002, -1.5718,  0.7656,  0.1405, -0.1569,  0.1042, -1.7913], [-0.6557,  0.0921, -0.9204, 
-0.7958,  0.3972, -0.1716,  0.4395, 0.5797,  0.1310,  0.9257], [-0.4573,  0.0659,  1.0679, -2.5354,  0.9847,  
0.3873, -0.6932, 0.0622, -0.3317, -0.1677], [ 0.3002,  0.9634,  0.3084, -0.8166,  0.7265,  0.7259, -2.4951, 1.7804,
1.1778,  1.0674], [-1.1359, -0.9332, -0.2955, -0.6292,  1.2434,  0.2434, -0.4777, -0.2757, -2.2602,  1.1969], [ 
0.0868, -0.6067,  0.7056, -0.2376,  0.3013,  0.1609, -0.6578, 0.8723,  1.3747,  0.1010], [-0.0177, -0.6365,  
0.0291,  0.8604,  1.7864,  0.4548,  1.5081, 0.2241, -0.5594, -0.8814], [ 1.4523, -0.6183, -1.9457, -0.4063, 
-0.1177, -0.1913,  0.4695, -0.7076, -0.3201,  0.6066], [-0.9230, -1.8305,  1.9993, -0.1577, -0.8544,  0.6160, 
-1.2820, 0.6445, -0.0412, -1.1240], [-0.1557,  1.1015,  0.2117,  1.1210, -0.9556, -0.3221, -0.3534, -0.1889,  
0.3864,  0.0268]],  [[ 0.1091, -0.7125,  0.2156,  0.3787, -1.6724,  0.6257, -0.1287, -0.1973,  0.1717, -0.5969], 
[-0.7402, -1.4093,  1.6512, -0.6289, -0.3070,  1.2325,  0.8291, -0.6882,  0.7387, -0.8708], [-0.2889, -1.4585,  
0.6022,  0.0169,  0.6015,  0.7624,  0.0133, -0.2833,  0.1824,  1.2131], [ 0.7378, -0.0402, -0.5133,  1.0691, 
-2.0410,  1.5720,  1.8389, -0.7687,  0.0430,  0.7435], [-0.0639, -0.8723, -2.0178,  0.1605,  1.4724,  1.1338, 
-0.8728, 1.4598,  0.3518, -1.7153], [ 0.6932,  0.7552,  1.0866,  0.6799,  0.3415, -0.0313, -0.2739, 2.1351, 
-0.6373, -0.6382], [ 0.0577, -0.0601,  0.0730, -0.2478, -1.1990, -0.1654,  1.7734, -0.5398, -0.4222,  0.2541], [ 
0.3999,  0.1831,  0.0746, -1.9693, -0.6570,  0.8493, -1.4996, 2.5116,  0.6886, -0.2006], [-0.5791,  0.0260, 
-0.3679, -1.1636, -0.2595,  0.9678, -1.4233, 0.8906,  0.3217, -1.1610], [-1.3476, -0.5151, -0.0477, -1.4329, 
-0.9107,  0.3661,  1.0603, -0.1931,  0.2535,  0.3719]]],   [[[-0.0433, -0.8572, -0.0829,  1.6075,  0.6515, -0.7278,
1.1583, -1.3607,  0.6071,  3.5932], [-1.6341, -2.5069, -0.2012,  0.4425,  0.5835,  1.4518,  0.2244, -0.0910, 
-0.4293,  1.2552], [-1.8680, -0.0931, -0.9054,  0.3668,  1.7401,  0.5720, -0.1235, 0.1858, -0.3974, -1.4302], 
[-0.5246,  0.6958, -0.6283, -0.0670, -1.6133,  0.7200, -0.6662, 0.9329, -0.3030, -0.8778], [-0.2268, -0.3574, 
-0.5492,  0.9996,  0.3966,  0.9895, -1.0842, 0.9693, -0.4399, -1.3256], [-0.7925,  0.4076, -0.5718, -1.5846, 
-0.2463,  0.9344, -0.0562, 1.0819,  0.4371,  0.3513], [ 0.2209, -1.8197, -0.2579, -0.4361,  2.1249,  1.0353, 
-0.9518, -0.6932,  0.2122, -0.1555], [-0.4044,  1.0599,  1.6030,  0.4628,  0.7142, -0.0272, -0.6217, -0.0647,  
0.1529,  0.3819], [-0.6737,  0.7189,  2.1411, -0.4039,  0.5217, -0.5200, -0.5558, -1.2184, -0.2036, -2.0343], [ 
0.2590, -1.1714,  0.1658,  1.0683, -0.0461, -0.7531, -0.0727, -0.0487, -1.1324, -0.3329]],  [[ 0.6310,  1.1288,  
2.1890,  0.3604,  0.8889,  0.0961, -0.9582, -0.4659, -0.9878,  0.4389], [ 0.0259,  0.9641,  1.2782, -0.3470,  
0.2701,  1.0423,  0.9026, -1.0581,  0.8089, -0.4029], [-0.4496,  0.9056,  0.0283,  0.5091,  0.0213,

In [38]:
print("All users of the initializer:", model.graph.initializers["weight"].uses())

All users of the initializer: (Usage(node=Node(name='node_mul', domain='', op_type='Mul', inputs=(SymbolicTensor(name='add', type=Tensor(FLOAT), shape=Shape([2, 3, 10, 10]), producer='node_add', index=0), SymbolicTensor(name='weight', type=Tensor(FLOAT), shape=Shape([2, 3, 10, 10]), const_value={TorchTensor(...)})), attributes={}, overload='', outputs=(SymbolicTensor(name='mul', type=Tensor(FLOAT), shape=Shape([2, 3, 10, 10]), producer='node_mul', index=0),), version=23, doc_string=None), idx=1),)


## Verify model outputs

https://github.com/justinchuby/model-explorer-onnx

In [39]:
from torch.onnx.verification import verify_onnx_program

from model_explorer_onnx.torch_utils import save_node_data_from_verification_info

verification_infos = verify_onnx_program(onnx_program, compare_intermediates=True)

# Produce node data for Model Explorer for visualization
save_node_data_from_verification_info(
    verification_infos, onnx_program.model, model_name="model"
)


In [40]:
!onnxvis model.onnx --node_data_paths=model_max_abs_diff.json,model_max_rel_diff.json

Loading extensions...
I0000 00:00:1767645930.860103  250391 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

Loaded 9 adapters:
 - TFLite adapter (Flatbuffer)
 - TFLite adapter (MLIR)
 - TF adapter (MLIR)
 - TF adapter (direct)
 - GraphDef adapter
 - Pytorch adapter (exported program)
 - MLIR adapter
 - ONNX adapter
 - JSON adapter

Starting Model Explorer server at:
http://localhost:8080/?data=%7B%22models%22%3A%20%5B%7B%22url%22%3A%20%22/home/justinchu/dev/talk-torch-onnx-apis-architecture/src/model.onnx%22%7D%5D%2C%20%22nodeData%22%3A%20%5B%22/home/justinchu/dev/talk-torch-onnx-apis-architecture/src/model_max_abs_diff.json%22%2C%20%22/home/justinchu/dev/talk-torch-onnx-apis-architecture/src/model_max_rel_diff.json%22%5D%2C%20%22nodeDataTargets%22%3A%20%5B%22%22%2C%20%22%22%5D%7D

Press Ctrl+

## Multiple ways to represent dynamic shapes

